# To Bleach or Not to Bleach? 

## Quickstart Notebook

**Goal:** identify drivers of coral bleaching and predict the percentage of coral that is bleached (`Percent_Bleached`).  
**Dataset:** csv file with site-level (aka "location"/coral reef) grouping by `Site_ID`.

### Cleaning

*Oh no, the data seems to be corrupted!* It looks like the people from the fossil fuel industry have been tampering with the data. There are some missing values, some outliers, and invalid/duplicate rows. **Start by cleaning it up!**


Cleaning steps:
1. Find outliers
2. Find missing values
3. Remove outliers
4. Remove NaNs


In [1]:
import os
import pandas as pd

filepath_train = r"../data/coral_students.csv"
df = pd.read_csv(filepath_train)

df.head()

,Sample_ID,Site_ID,ClimSST,Temperature_Kelvin,Temperature_Mean,Temperature_Minimum,Temperature_Maximum,Temperature_Kelvin_Standard_Deviation,Windspeed,SSTA,...,TSA_DHW,TSA_DHW_Standard_Deviation,TSA_DHWMax,TSA_DHWMean,Depth_m,Distance_to_Shore,Exposure,Turbidity,Cyclone_Frequency,Percent_Bleached
0,10274495.0,12082.0,301.65,303.50,299.79,293.35,305.54,2.52,2.0,0.49,...,1.27,0.74,6.05,0.22,8.3,8311.0,exposed,0.0586,56.583448,4.76
1,10274496.0,12083.0,299.31,300.84,299.75,293.68,305.44,2.54,6.0,-0.42,...,1.20,0.93,10.39,0.27,14.9,10747.0,exposed,0.0543,52.842523,21.88
2,10274497.0,12084.0,300.56,302.65,299.81,293.35,305.47,2.50,5.0,0.36,...,2.71,0.83,7.18,0.23,10.7,9396.0,exposed,0.0571,56.583448,19.66
3,10274498.0,12085.0,299.75,302.43,299.81,293.35,305.47,2.50,7.0,0.54,...,3.60,0.83,7.18,0.23,7.6,9408.0,exposed,0.0571,56.583448,28.03
4,10274499.0,12086.0,297.65,295.69,299.81,293.35,305.47,2.50,7.0,-0.91,...,0.00,0.83,7.18,0.23,10.0,9362.0,exposed,0.0571,56.583448,2.75


In [2]:
import pandas as pd
from typing import Iterable, Dict, Tuple, Literal
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

Strategy = Literal["impute", "drop"]

def iqr_mask(df: pd.DataFrame, columns: Iterable[str], k: float = 1.5) -> Dict[str, pd.Series]:
    """
    Return a boolean mask per selected column: True where value is an IQR outlier.
    k=1.5 -> Tukey 'mild' outliers; use 3.0 for 'extreme'.
    """
    masks: Dict[str, pd.Series] = {}
    for col in columns:
        s = pd.to_numeric(df[col], errors="coerce")
        q1 = s.quantile(0.25)
        q3 = s.quantile(0.75)
        iqr = q3 - q1
        if pd.isna(iqr) or iqr == 0:
            masks[col] = pd.Series(False, index=df.index)
        else:
            lower = q1 - k * iqr
            upper = q3 + k * iqr
            masks[col] = (s < lower) | (s > upper)
    return masks

def clean_outliers(
    df: pd.DataFrame,
    outlier_columns: Iterable[str],
    *,
    k_iqr: float = 1.5,
    strategy: Strategy = "impute",
    n_neighbors: int = 5,
    weights: str = "distance",
    output_csv: str | None = None,
) -> Tuple[pd.DataFrame, pd.Series]:
    """
    Pipeline:
      1) Mark IQR outliers in `outlier_columns` -> NaN
      2) strategy == 'impute': KNN-impute numeric columns (scaled)
         strategy == 'drop'  : drop rows having NaN in any of `outlier_columns`
      3) Drop any remaining NaNs (whole-row drop)
    Returns: (clean_df, outlier_row_mask_before_cleaning)
    """
    df_work = df.copy()

    # 1) Detect outliers & set to NaN (only in requested columns)
    masks = iqr_mask(df_work, outlier_columns, k=k_iqr)
    mask_df = pd.DataFrame(masks, index=df_work.index) if masks else pd.DataFrame(index=df_work.index)
    any_outlier = mask_df.any(axis=1) if not mask_df.empty else pd.Series(False, index=df_work.index)

    for col, m in masks.items():
        df_work.loc[m, col] = pd.NA

    # 2) Clean by strategy
    if strategy == "impute":
        num_cols = df_work.select_dtypes(include="number").columns.tolist()
        if num_cols:
            scaler = StandardScaler()
            imputer = KNNImputer(n_neighbors=n_neighbors, weights=weights)

            X_num = df_work[num_cols].astype(float).values
            X_scaled = scaler.fit_transform(X_num)
            X_imputed_scaled = imputer.fit_transform(X_scaled)
            X_imputed = scaler.inverse_transform(X_imputed_scaled)

            df_work[num_cols] = X_imputed
        # else: nothing to impute, proceed
    elif strategy == "drop":
        df_work = df_work.dropna(subset=list(outlier_columns), how="any")
    else:
        raise ValueError("strategy must be 'impute' or 'drop'")

    # 3) Drop any remaining NaNs anywhere (whole row)
    df_clean = df_work.dropna(axis=0, how="any").reset_index(drop=True)

    if output_csv:
        df_clean.to_csv(output_csv, index=False)

    return df_clean, any_outlier


In [3]:
print(df.count())

df.isna().sum()

Sample_ID                                4445
Site_ID                                  4445
ClimSST                                  4445
Temperature_Kelvin                       4445
Temperature_Mean                         4445
Temperature_Minimum                      4445
Temperature_Maximum                      4445
Temperature_Kelvin_Standard_Deviation    4445
Windspeed                                4445
SSTA                                     4445
SSTA_Standard_Deviation                  4445
SSTA_Mean                                4445
SSTA_Minimum                             4445
SSTA_Maximum                             4445
SSTA_Frequency                           4445
SSTA_Frequency_Standard_Deviation        4445
SSTA_FrequencyMax                        4445
SSTA_FrequencyMean                       4445
SSTA_DHW                                 4445
SSTA_DHW_Standard_Deviation              4445
SSTA_DHWMax                              4445
SSTA_DHWMean                      

Sample_ID                                  1
Site_ID                                    1
ClimSST                                    1
Temperature_Kelvin                         1
Temperature_Mean                           1
Temperature_Minimum                        1
Temperature_Maximum                        1
Temperature_Kelvin_Standard_Deviation      1
Windspeed                                  1
SSTA                                       1
SSTA_Standard_Deviation                    1
SSTA_Mean                                  1
SSTA_Minimum                               1
SSTA_Maximum                               1
SSTA_Frequency                             1
SSTA_Frequency_Standard_Deviation          1
SSTA_FrequencyMax                          1
SSTA_FrequencyMean                         1
SSTA_DHW                                   1
SSTA_DHW_Standard_Deviation                1
SSTA_DHWMax                                1
SSTA_DHWMean                               1
TSA       

In [4]:
import pandas as pd

cols = [
    "Temperature_Kelvin", "Temperature_Mean", "Temperature_Minimum",
    "Temperature_Maximum", "Temperature_Kelvin_Standard_Deviation",
    "Windspeed", "Depth_m", "Distance_to_Shore"
]

# 1) Get per-column outlier masks (dict[str, Series])
masks = iqr_mask(df, cols, k=1.5)

# 2) Build a flagged DataFrame with <col>_is_outlier columns
df_flagged = df.copy()
for col, m in masks.items():
    df_flagged[f"{col}_is_outlier"] = m

# 3) Combine into a single 'any_outlier' flag
if masks:  # protect against empty list
    df_flagged["any_outlier"] = pd.concat(masks.values(), axis=1).any(axis=1)
else:
    df_flagged["any_outlier"] = False

# 4) Inspect outlier rows
outlier_rows = df_flagged[df_flagged["any_outlier"]]
print(outlier_rows.count())


Sample_ID                                           594
Site_ID                                             594
ClimSST                                             594
Temperature_Kelvin                                  594
Temperature_Mean                                    594
Temperature_Minimum                                 594
Temperature_Maximum                                 594
Temperature_Kelvin_Standard_Deviation               594
Windspeed                                           594
SSTA                                                594
SSTA_Standard_Deviation                             594
SSTA_Mean                                           594
SSTA_Minimum                                        594
SSTA_Maximum                                        594
SSTA_Frequency                                      594
SSTA_Frequency_Standard_Deviation                   594
SSTA_FrequencyMax                                   594
SSTA_FrequencyMean                              

In [5]:
clean_df, outlier_rows = clean_outliers(
    df,
    cols,
    k_iqr=1.5,
    strategy="drop",
    output_csv="../data/cleaned_data_dropped.csv",
)
clean_df.isna().sum()

Sample_ID                                0
Site_ID                                  0
ClimSST                                  0
Temperature_Kelvin                       0
Temperature_Mean                         0
Temperature_Minimum                      0
Temperature_Maximum                      0
Temperature_Kelvin_Standard_Deviation    0
Windspeed                                0
SSTA                                     0
SSTA_Standard_Deviation                  0
SSTA_Mean                                0
SSTA_Minimum                             0
SSTA_Maximum                             0
SSTA_Frequency                           0
SSTA_Frequency_Standard_Deviation        0
SSTA_FrequencyMax                        0
SSTA_FrequencyMean                       0
SSTA_DHW                                 0
SSTA_DHW_Standard_Deviation              0
SSTA_DHWMax                              0
SSTA_DHWMean                             0
TSA                                      0
TSA_Standar